In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import json
import math
import random
import statistics
import matplotlib.patches as patches
import matplotlib.colors as mcolors


# euclidean distance tracking

In [ ]:
def get_unique_trees(tree_data):
    # remove overlapping bounding boxes

    pred_boxes = tree_data["pred_boxes"]

    return get_overlapping_boxes(pred_boxes)



def get_overlapping_boxes(pred_boxes):
    overlapping_boxes = []

    for i in range(len(pred_boxes)):
        for j in range(i+1, len(pred_boxes)):
            box1 = pred_boxes[i]
            box2 = pred_boxes[j]

            # Check for overlap
            if (box1[0] < box2[2] and box1[2] > box2[0] and
                box1[1] < box2[3] and box1[3] > box2[1]):
                # Overlap found, add both boxes

                if calculate_overlap(box1, box2) > 0.1:

                    overlapping_boxes.append(box1)
                    overlapping_boxes.append(box2)

    return overlapping_boxes

def get_lowest_box(boxes):
    if not boxes:
        return None
    
    lowest_box = boxes[0]
    for box in boxes[1:]:
        if box[3] < lowest_box[3]:
            lowest_box = box
    return lowest_box


def calculate_overlap(box1, box2):
    # Calculate overlap area
    overlap_width = min(box1[2], box2[2]) - max(box1[0], box2[0])
    overlap_height = min(box1[3], box2[3]) - max(box1[1], box2[1])

    if overlap_width <= 0 or overlap_height <= 0:
        return 0

    overlap_area = overlap_width * overlap_height

    # Calculate area of each box
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate overlapping percentage
    overlap_percentage_box1 = overlap_area / area_box1
    overlap_percentage_box2 = overlap_area / area_box2

    return max(overlap_percentage_box1, overlap_percentage_box2)

def get_highest_overlap_percentage_boxes(pred_boxes):
    overlapping_boxes = []
    max_overlap_percentage = 0

    for i in range(len(pred_boxes)):
        for j in range(i + 1, len(pred_boxes)):
            box1 = pred_boxes[i]
            box2 = pred_boxes[j]

            overlap_percentage = calculate_overlap(box1, box2)

            if overlap_percentage > 0 and overlap_percentage < 1:
                if overlap_percentage > max_overlap_percentage:
                    overlapping_boxes = [(box1, box2)]
                    max_overlap_percentage = overlap_percentage
                elif overlap_percentage == max_overlap_percentage:
                    overlapping_boxes.append((box1, box2))

    return overlapping_boxes, max_overlap_percentage

def process_data(data):
    while(len(get_unique_trees(data)) > 0):
        unique_trees = get_unique_trees(data)

        test = get_highest_overlap_percentage_boxes(unique_trees)

        if(test[0][0][0][3]>test[0][0][1][3]):
            tree_to_remove = test[0][0][1]
        else:
            tree_to_remove = test[0][0][0]

        for index, tree in enumerate(data["pred_boxes"]):
            if(tree[0] == tree_to_remove[0] and tree[1] == tree_to_remove[1] and tree[3] == tree_to_remove[3]):
                data["pred_boxes"].pop(index)
                data["pred_keypoints"].pop(index)
                break

    return data


In [ ]:
folder_path = './assets/undistorted_05/eastbound'
data_path = "./assets/annotated_05/eastbound/"

files = os.listdir(folder_path)

def getNumber(x):
    num_w_ext = x.split('_')[2]
    num = num_w_ext.split('.')[0]
    return num

sorted_files = sorted(files, key=lambda x: int(getNumber(x)))

normal_images = []
transformed_images = []

for img_source in sorted_files:
    image = cv2.imread(f"{folder_path}/{img_source}")
    normal_images.append(image)

In [ ]:
img_i = 10
colors = list(mcolors.CSS4_COLORS.keys())
random.shuffle(colors)


name = sorted_files[img_i].replace('png', 'json')

with open(data_path + name, 'r') as json_file:
# Load the JSON data
    unprocessed_data = json.load(json_file)


data = process_data(unprocessed_data)

# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(normal_images[img_i])

# Create a Rectangle patch

for index, box in enumerate(data["pred_boxes"]):
    x, y, w, h = data["pred_boxes"][index]

    w = w-x
    h = h-y

    bbox = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=colors[index], facecolor='none')
    ax.scatter(x + (w / 2), y + (h/2), color='r', s=50)
    ax.add_patch(bbox)

# Show the plot
plt.show()

In [ ]:
def calculate_eucl_dist(point_1, point_2):
    x1, y1 = point_1
    x2, y2 = point_2

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)


def detect_obj(data_1, data_2):
    objs_1 = []
    objs_2 = []

    for index, box in enumerate(data_1["pred_boxes"]):
        x, y, x_2, y_2 = box

        w = x_2-x
        h = y_2-y

        centroid = (x + (w / 2), y + (h/2))

        objs_1.append(centroid)

    for index, box in enumerate(data_2["pred_boxes"]):
        x, y, x_2, y_2 = box

        w = x_2-x
        h = y_2-y

        centroid = (x + (w / 2), y + (h/2))

        objs_2.append(centroid)



    trees = []

    for index_obj_1, centroid_1 in enumerate(objs_1):
        for index_obj_2, centroid_2 in enumerate(objs_2):

            dist = calculate_eucl_dist(centroid_1, centroid_2)


            obj_map = (index_obj_1, index_obj_2, dist)
            trees.append(obj_map)
    
        
        # maps.append((index_obj_2, index_lowest_obj_1))

    
    obj_maps = []
    

    sorted_trees = sorted(trees, key=lambda x: x[2])


    while (len(sorted_trees)>0):
        if sorted_trees[0][2] < 70:
            first_tree = sorted_trees[0]

            sorted_trees.pop(0)

            indexes_to_pop = []

            for index, tree in enumerate(sorted_trees):
                if first_tree[0] == tree[0]:
                    indexes_to_pop.append(index)
                if first_tree[1] == tree[1]:
                    indexes_to_pop.append(index)


            indexes_to_pop.reverse()

            for index in indexes_to_pop:
                sorted_trees.pop(index)

            obj_maps.append(first_tree)
        else:
            sorted_trees.pop(0)


    return obj_maps
    

colors = list(mcolors.CSS4_COLORS.keys())
random.shuffle(colors)

prev_maps = {}

universal_tree_ids = []

# for i in range(0,5):
def update(i):
    global prev_maps
    global universal_tree_ids

    ax1.clear()

    name = sorted_files[i].replace('png', 'json')

    with open(data_path + name, 'r') as json_file:
    # Load the JSON data
        unprocessed_data = json.load(json_file)




    data = process_data(unprocessed_data)

    if i == 0:
        for index,pred_box in enumerate(data["pred_boxes"]):
            universal_tree_ids.append({
                'color': colors[index],
                'hist': {
                    i: index
                }
            })

            prev_maps[str(index)] = index

    else:
        name = sorted_files[i - 1].replace('png', 'json')

        with open(data_path + name, 'r') as json_file:
        # Load the JSON data
            unprocessed_data = json.load(json_file)


        data_prev = process_data(unprocessed_data)

        obj_maps = detect_obj(data_prev, data)

        temp = {}

        for tree in obj_maps:


            if str(tree[0]) in prev_maps:
                universal_id = prev_maps[str(tree[0])]

                temp[str(tree[1])] = universal_id

                universal_tree_ids[universal_id]["hist"][i] = tree[1]
        
        prev_maps = temp


    
    # fig, ax = plt.subplots()

    ax1.imshow(normal_images[i])

    # Create a Rectangle patch

    for index, box in enumerate(data["pred_boxes"]):
        if str(index) in prev_maps:
            color = universal_tree_ids[prev_maps[str(index)]]['color']
        else:
            color = colors[i]

            universal_tree_ids.append({
                'color': colors[i],
                'hist': {
                    i: index
                }
            })

            prev_maps[str(index)] = len(universal_tree_ids) - 1



        x, y, w, h = box

        w = w-x
        h = h-y

        bbox = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor=color, facecolor='none')
        ax1.scatter(x + (w / 2), y + (h/2), color='r', s=50)
        ax1.add_patch(bbox)

    # Show the plot
    # plt.show()
    return ax1


plt.clf()

fig, (ax1) = plt.subplots(1, 1, figsize=(6, 4))

ani = animation.FuncAnimation(fig, update, frames=50, interval=150)


HTML(ani.to_jshtml())

print(universal_tree_ids)


# triangulation

In [ ]:
import cv2
import numpy as np

name = sorted_files[0].replace('png', 'json')

with open(data_path + name, 'r') as json_file:
# Load the JSON data
    unprocessed_data_1 = json.load(json_file)

name = sorted_files[0].replace('png', 'json')

with open(data_path + name, 'r') as json_file:
# Load the JSON data
    unprocessed_data_2 = json.load(json_file)

# Load images
image1 = normal_images[1]
image2 = normal_images[2]


test_1 = unprocessed_data_1["pred_keypoints"][1][0]
test_2 = unprocessed_data_2["pred_keypoints"][2][0]

P1 = (test_1[0], test_1[1])
P2 = (test_2[0], test_2[1])

# Perform feature detection and matching
orb = cv2.ORB_create()
keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(descriptors1, descriptors2)

# Extract matched keypoints
matched_keypoints1 = np.float32([keypoints1[match.queryIdx].pt for match in matches])
matched_keypoints2 = np.float32([keypoints2[match.trainIdx].pt for match in matches])

# Triangulate points
points4D = cv2.triangulatePoints(P1, P2, matched_keypoints1.T, matched_keypoints2.T)

# Convert homogeneous coordinates to Cartesian coordinates
points3D = cv2.convertPointsFromHomogeneous(points4D.T)

# Visualize the 3D points
for point in points3D:
    x, y, z = point[0]
    print(f"3D Point: ({x}, {y}, {z})")

In [ ]:
# Load camera calibration data (intrinsic parameters)
K = np.load('intrinsic_parameters.npy')

# Assume relative translation between camera positions (from vehicle motion)
t = np.array([0.3, 0, 0])  # Adjust baseline according to vehicle motion

# Compute projection matrices P1 and P2
P1 = np.hstack((K, np.zeros((3, 1))))
P2 = np.hstack((K, np.array([[t[0]], [0], [0]])))  # Assuming no rotation between images

print("Projection Matrix P1:")
print(P1)
print("\nProjection Matrix P2:")
print(P2)

In [ ]:
import cv2
import numpy as np


image1 = normal_images[1]
image2 = normal_images[2]


# Perform feature detection and matching
orb = cv2.ORB_create()
keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(descriptors1, descriptors2)

# Extract matched keypoints
matched_keypoints1 = np.float32([keypoints1[match.queryIdx].pt for match in matches])
matched_keypoints2 = np.float32([keypoints2[match.trainIdx].pt for match in matches])

# Triangulate points
points4D = cv2.triangulatePoints(P1, P2, matched_keypoints1.T, matched_keypoints2.T)

# Convert homogeneous coordinates to Cartesian coordinates
points3D = cv2.convertPointsFromHomogeneous(points4D.T)

# Visualize the 3D points
for point in points3D:
    x, y, z = point[0]
    print(f"3D Point: ({x}, {y}, {z})")

In [ ]:
import cv2
import numpy as np

image1 = normal_images[1]
image2 = normal_images[2]

name = sorted_files[1].replace('png', 'json')

with open(data_path + name, 'r') as json_file:
# Load the JSON data
    unprocessed_data_1 = json.load(json_file)

name = sorted_files[2].replace('png', 'json')

with open(data_path + name, 'r') as json_file:
# Load the JSON data
    unprocessed_data_2 = json.load(json_file)

# List of objects detected in image1 and image2
objects_image1 = unprocessed_data_1["pred_boxes"]  # List of bounding boxes or ROIs of detected objects in image1
objects_image2 = unprocessed_data_2["pred_boxes"]  # List of bounding boxes or ROIs of detected objects in image2

# Initialize an empty list to store the 3D positions of each tracked object
tracked_objects_3D = []

for object_bbox1, object_bbox2 in zip(objects_image1, objects_image2):

    try:
        # Extract the region of interest (ROI) containing the detected object in both images
        object_roi1 = image1[int(object_bbox1[1]):int(object_bbox1[3]), int(object_bbox1[0]):int(object_bbox1[2])]
        object_roi2 = image2[int(object_bbox2[1]):int(object_bbox2[3]), int(object_bbox2[0]):int(object_bbox2[2])]

        plt.imshow(object_roi1)
        plt.show()
        plt.imshow(object_roi2)
        plt.show()

        x1, y1, x2, y2 = object_bbox1
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        im_copy = np.copy(image1)
        color = (0, 255, 0)  # Green color
        thickness = 2  # Line thickness
        cv2.rectangle(im_copy, (x1, y1), (x2, y2), color, thickness)
        plt.show(im_copy)
        plt.show()

        # Perform feature detection and matching within the object ROIs
        orb = cv2.ORB_create()
        keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
        keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(descriptors1, descriptors2)

        # Extract matched keypoints within the object ROIs
        matched_keypoints1 = np.float32([keypoints1[match.queryIdx].pt for match in matches])
        matched_keypoints2 = np.float32([keypoints2[match.trainIdx].pt for match in matches])

        # Triangulate matched keypoints
        points4D = cv2.triangulatePoints(P1, P2, matched_keypoints1.T, matched_keypoints2.T)

        # Convert homogeneous coordinates to Cartesian coordinates
        points3D = cv2.convertPointsFromHomogeneous(points4D.T)

        # Add the 3D position of the tracked object to the list
        tracked_objects_3D.append(points3D)
    except Exception as e:
        print('error: ',e)

# Visualize the 3D positions of tracked objects
for object_idx, object_3D in enumerate(tracked_objects_3D):
    for point in object_3D:
        x, y, z = point[0]
        print(f"Object {object_idx+1}, 3D Point: ({x}, {y}, {z})")


In [ ]:
image1 = normal_images[1]
image2 = normal_images[2]        


orb = cv2.ORB_create()
keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(descriptors1, descriptors2)

# Extract matched keypoints within the object ROIs
matched_keypoints1 = np.float32([keypoints1[match.queryIdx].pt for match in matches])
matched_keypoints2 = np.float32([keypoints2[match.trainIdx].pt for match in matches])

# Draw match

# Draw matches
result_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, None)

# Convert result image to RGB format for Matplotlib
result_image_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

# Display the result using Matplotlib
plt.figure(figsize=(12, 6))
plt.imshow(result_image_rgb)
plt.axis('off')
plt.show()


In [ ]:
def is_point_in_bbox(point, bbox):
    x, y = point
    xmin, ymin, width, height = bbox

    if xmin <= x <= xmin + width and ymin <= y <= ymin + height:
        return True
    else:
        return False
    
def get_data(img_id):
    name = sorted_files[img_id].replace('png', 'json')

    with open(data_path + name, 'r') as json_file:
    # Load the JSON data
        unprocessed_data = json.load(json_file)

    return unprocessed_data


def get_possible_tree_matches(image_1, pred_boxes_1, image_2, pred_boxes_2):
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(image_1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image_2, None)

    # Define FLANN parameters for descriptor matching
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    # Create FLANN matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    # Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)




    tree_matches = []

    # bbox = unprocessed_data_1["pred_boxes"][requested_bbox]
    for index_bbox, bbox in enumerate(pred_boxes_1):

        keypoints1_in_bbox = [kp for kp in keypoints1 if bbox[0] <= kp.pt[0] <= bbox[2] and
                            bbox[1] <= kp.pt[1] <= bbox[3]]

        matches_in_bbox = []
        for match in good_matches:
            kp1 = keypoints1[match.queryIdx]
            kp2 = keypoints2[match.trainIdx]
            if kp1 in keypoints1_in_bbox:
                matches_in_bbox.append(match)
    

        for index, bbox in enumerate(pred_boxes_2):

            matches_a = 0

            for match in matches_in_bbox:
                if(is_point_in_bbox(keypoints2[match.trainIdx].pt, bbox)):
                    matches_a+=1

            tree_matches.append((index_bbox, index, matches_a))


    # print(tree_matches)
    return tree_matches


In [ ]:


# img_id_1 = 5
# img_id_2 = img_id_1 + 1

# image_1 = normal_images[img_id_1]
# image_2 = normal_images[img_id_2]

# unprocessed_data_1 = get_data(img_id_1)
# unprocessed_data_2 = get_data(img_id_2)

# pred_boxes_1 = unprocessed_data_1["pred_boxes"]
# pred_boxes_2 = unprocessed_data_2["pred_boxes"]

# tree_matches = get_possible_tree_matches(image_1,pred_boxes_1, image_2, pred_boxes_2)

# sorted_trees = sorted(tree_matches, key=lambda x: x[2])



# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,6))

# color_index = 0
def get_def_tree_matches( sorted_trees):
    def_tree_matches = []
    while len(sorted_trees) > 0:

        temp_tree = sorted_trees[-1]

        if(temp_tree[2]>1):
            def_tree_matches.append(temp_tree)

            # print(temp_tree, colors[color_index])
            # print(sorted_trees)
            sorted_trees.pop()

            filtered_tree_matches = [tree for tree in sorted_trees if tree[0] != temp_tree[0] and tree[1] != temp_tree[1]]

            # print(filtered_tree_matches)

            # bbox = pred_boxes_1[temp_tree[0]]

            # rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3]- bbox[1], linewidth=1, edgecolor=colors[color_index], facecolor='none')

            # ax1.add_patch(rect)
            # ax1.imshow(image_1)
            # ax1.axis('off')


            # bbox = pred_boxes_2[temp_tree[1]]
            # rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3]- bbox[1], linewidth=1, edgecolor=colors[color_index], facecolor='none')

            # ax2.add_patch(rect)
            # ax2.imshow(image_2)
            # ax2.axis('off')

            sorted_trees = filtered_tree_matches

            # color_index += 1
            # break
        else:
            sorted_trees.pop()

    return def_tree_matches

# plt.show()

In [ ]:
prev_maps = {}

universal_tree_ids = []

colors = list(mcolors.CSS4_COLORS.keys())
random.shuffle(colors)


def update_2(i):
    global prev_maps
    global universal_tree_ids

    data = get_data(i)

    ax1.clear()
    ax1.imshow(normal_images[i])
    ax1.axis('off')

    if i == 0:
        for index, pred_box in enumerate(data["pred_boxes"]):
            universal_tree_ids.append({
                'color': colors[index],
                'hist': {
                    i: index
                }
            })

            prev_maps[str(index)] = index

            bbox = pred_box
            rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3]- bbox[1], linewidth=1, edgecolor=colors[index], facecolor='none')
            ax1.add_patch(rect)

    else:
        # print(universal_tree_ids)
        # print(prev_maps)
        img_id_1 = i-1
        img_id_2 = img_id_1 + 1

        image_1 = normal_images[img_id_1]
        image_2 = normal_images[img_id_2]

        unprocessed_data_1 = get_data(img_id_1)
        unprocessed_data_2 = get_data(img_id_2)

        pred_boxes_1 = unprocessed_data_1["pred_boxes"]
        pred_boxes_2 = unprocessed_data_2["pred_boxes"]

        tree_matches = get_possible_tree_matches(image_1,pred_boxes_1, image_2, pred_boxes_2)

        sorted_trees = sorted(tree_matches, key=lambda x: x[2])

        def_tree_matches = get_def_tree_matches(sorted_trees)



        temp = {}

        for tree in def_tree_matches:
            if str(tree[0]) in prev_maps:
                universal_id = prev_maps[str(tree[0])]

                temp[str(tree[1])] = universal_id

                universal_tree_ids[universal_id]["hist"][i] = tree[1]

                bbox = pred_boxes_2[tree[1]]
                rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3]- bbox[1], linewidth=1, edgecolor=universal_tree_ids[universal_id]["color"], facecolor='none')
                ax1.add_patch(rect)

            else:
                print('else')
                universal_tree_ids.append({
                    'color': colors[len(universal_tree_ids)],
                    'hist': {
                        i: tree[1]
                    }
                })

                temp[str(tree[1])] = len(universal_tree_ids) -1
            
                # print(tree, colors[len(universal_tree_ids)])


                # 
                bbox = pred_boxes_2[tree[1]]
                rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3]- bbox[1], linewidth=1, edgecolor=colors[len(universal_tree_ids)], facecolor='none')
                ax1.add_patch(rect)
                
        
        prev_maps = temp

    





plt.clf()

fig, (ax1) = plt.subplots(1, 1, figsize=(6, 4))

ani = animation.FuncAnimation(fig, update_2, frames=100, interval=150)


HTML(ani.to_jshtml())